<a href="https://colab.research.google.com/github/belleaud/StockDayScreener/blob/main/StocksDayScreener.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas-datareader
!pip install yfinance 
!pip install yahoo_fin
!pip install ta

In [ ]:
import pandas_datareader.data as reader
import datetime as dt
import yfinance as yf
from datetime import timedelta

start = dt.datetime.today() - timedelta(days=1)
end = dt.datetime.now()
symbol = 'AAPL'

df = yf.download(symbol, period = '5d')


In [ ]:
df = df.reset_index()
df['stock_id'] = 1
df['stock_id'] = df['stock_id'].astype(int)


In [4]:
import pandas as pd
from yahoo_fin import stock_info as si
import numpy as np
import ta
from datetime import date, timezone, timedelta

def get_stocktickers():
  # gather stock symbols from major US exchanges
  df1 = pd.DataFrame( si.tickers_sp500() )
  df2 = pd.DataFrame( si.tickers_nasdaq() )
  df3 = pd.DataFrame( si.tickers_dow() )
  df4 = pd.DataFrame( si.tickers_other() )
  df = pd.concat([df1,df2,df3,df4], ignore_index=True)
  df.rename(columns={0: 'Symbol'}, inplace=True)
  df.drop(df[(df['Symbol'].str.contains('\.|\-')==True)].index, inplace=True)

  return df

# symbols = get_stocktickers()

# Donne la liste des top gainers après filtrage de prix et volume moyen 3 mois.
def get_day_gainers(nb_gainers=100, minprice=2, maxprice=45, minavgvol=1000000):
  top = si.get_day_gainers(nb_gainers)
  top = top.iloc[:,0:7]
  top.drop(top[(top['Price (Intraday)'] <= minprice) & (top['Price (Intraday)'] >= maxprice)].index , inplace=True)
  top.drop(top[top['Avg Vol (3 month)'] <= minavgvol].index, inplace=True)

  return top

hier = date.today() - timedelta(days=1)
today = date.today().strftime("%Y/%m/%d")
gainers = get_day_gainers(nb_gainers=100, minprice=2, maxprice=45, minavgvol=1000000)

for symbol in gainers.Symbol:
  print(symbol)
  df_1 = si.get_data(symbol, start_date=hier, interval='1m', index_as_date=True)
  df_1['FastEMA'] = ta.trend.ema_indicator(df_1['close'], window=21)
  df_1['SlowEMA'] = ta.trend.ema_indicator(df_1['close'], window=50)
  df_1['STOCHk'] = ta.momentum.stoch(df_1['high'], df_1['low'], df_1['close'],fillna=True)      
  df_1['STOCHd'] = ta.momentum.stoch_signal(df_1['high'], df_1['low'], df_1['close'], fillna=True)
  df_1.drop(columns=['ticker'], inplace=True)  
  df_1.index.names = ['TIME']
  df = df_1.asfreq('3min')
  df.reset_index(inplace=True)
  df.dropna()

  break

df.head(10)



TPTX


,TIME,open,high,low,close,volume,FastEMA,SlowEMA,STOCHk,STOCHd
0,2022-06-03 13:30:00,73.779999,74.000000,73.730003,73.849998,4511101,NaN,NaN,44.443189,44.443189
1,2022-06-03 13:33:00,73.820000,73.860001,73.680000,73.700401,299204,NaN,NaN,6.375319,29.902099
2,2022-06-03 13:36:00,73.544998,73.629303,73.529999,73.620003,174007,NaN,NaN,19.149731,8.992874
3,2022-06-03 13:39:00,73.610001,73.620003,73.610001,73.614998,70656,NaN,NaN,18.084865,15.957297
4,2022-06-03 13:42:00,73.625000,73.650002,73.610001,73.635002,182617,NaN,NaN,22.341082,21.631712
5,2022-06-03 13:45:00,74.000000,74.000000,73.970001,73.970001,236146,NaN,NaN,93.617298,68.717677
6,2022-06-03 13:48:00,73.849998,73.900002,73.849998,73.870003,76868,NaN,NaN,72.341082,75.461550
7,2022-06-03 13:51:00,73.750000,73.760002,73.709999,73.709999,121373,73.776506,NaN,29.268701,44.457212
8,2022-06-03 13:54:00,73.669998,73.680000,73.639999,73.644997,125353,73.755861,NaN,8.973356,22.221787
9,2022-06-03 13:57:00,73.750000,73.800003,73.730003,73.764999,61292,73.751033,NaN,41.250072,33.333333


In [ ]:
# import libaries
import pandas as pd
from io import StringIO

# example data
text = """
TIME                 Open   High    Low     CLOSE   VOLUME
2019-10-31T09:15:00 235.70  236.50  234.40  234.55  1306585 
2019-10-31T10:15:00 234.55  235.05  233.40  234.00  765419      
2019-10-31T11:15:00 234.00  235.25  233.65  234.10  664682      
2019-10-31T12:15:00 234.10  235.75  234.00  235.70  676993      
2019-10-31T13:15:00 235.75  236.15  234.25  235.25  527381      
2019-10-31T14:15:00 235.25  235.80  227.10  227.45  2416364     
2019-10-31T15:15:00 227.50  228.00  227.05  227.70  613380      
2019-11-01T09:15:00 227.70  232.45  227.50  231.85  1844851     
2019-11-01T10:15:00 231.95  233.05  231.05  232.20  1000537     
2019-11-01T11:15:00 232.20  232.65  231.05  232.60  454966      
2019-11-01T12:15:00 232.60  233.50  231.40  231.70  569539      
2019-11-01T13:15:00 231.70  236.45  231.40  235.35  1388397 
"""

# create example df
df = pd.read_csv(StringIO(text), header=0, sep='\s+', parse_dates=['TIME'])

# melt data to get in the right form for resample and .ohlc()
df_melt = pd.melt(
    df, 
    id_vars=['TIME'], 
    value_vars=['Open', 'High', 'Low', 'CLOSE']
).drop(columns=['variable'])

# resample data by 2 hours and use origin=start to start with 
# the first available timestamp, in this case 09:15:00
result_df = (df_melt
    .set_index('TIME')
    .resample('3h', origin='start')
    .ohlc()
    .dropna()
)

In [ ]:
df


,TIME,Open,High,Low,CLOSE,VOLUME
0,2019-10-31 09:15:00,235.70,236.50,234.40,234.55,1306585
1,2019-10-31 10:15:00,234.55,235.05,233.40,234.00,765419
2,2019-10-31 11:15:00,234.00,235.25,233.65,234.10,664682
3,2019-10-31 12:15:00,234.10,235.75,234.00,235.70,676993
4,2019-10-31 13:15:00,235.75,236.15,234.25,235.25,527381
5,2019-10-31 14:15:00,235.25,235.80,227.10,227.45,2416364
6,2019-10-31 15:15:00,227.50,228.00,227.05,227.70,613380
7,2019-11-01 09:15:00,227.70,232.45,227.50,231.85,1844851
8,2019-11-01 10:15:00,231.95,233.05,231.05,232.20,1000537
9,2019-11-01 11:15:00,232.20,232.65,231.05,232.60,454966


In [ ]:
#info = si.get_company_info('AAPL')
#info = si.get_income_statement('AAPL', True)
#info = si.get_cash_flow('AAPL')
#info = si.get_day_gainers(100)
#info = si.get_day_most_active(10)
#info = si.get_undervalued_large_caps()
#info = si.get_data('AAPL')
#info = si.get_stats('AAPL')
#info = si.get_stats_valuation('AAPL')
#info = info.iloc[:1,:2]
#info.rename(columns={1: 'Marketcap'}, inplace=True)
#print(info['Marketcap'])
result_df.info()


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5 entries, 2019-10-31 09:15:00 to 2019-11-01 12:15:00
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   (value, open)   5 non-null      float64
 1   (value, high)   5 non-null      float64
 2   (value, low)    5 non-null      float64
 3   (value, close)  5 non-null      float64
dtypes: float64(4)
memory usage: 200.0 bytes
